# Code for modifying raw RDF ontology file to use for training Owl2Vec*
* Create 3 files, each containing relations of only one kind
* Remove test axioms to create train data

## Split by relation types (in rdfxml file)

In [ ]:
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [ ]:
onto_path.append("C:/Users/Abhishek/Desktop/Semester-8/IP/data/raw")

In [ ]:
for DATASET in ["GALEN"]:
  for split in ["1_1", "1_n", "n_n"]:
    if DATASET == "SNOMED":
      onto = get_ontology("snomed.owl").load(
        reload=True, only_local=True
        )
      splitter = "#"
    if DATASET == "GALEN":
      onto = get_ontology("galen.owl").load(reload=True, only_local=True)
      splitter = "#"
    if DATASET == "GO":
      onto = get_ontology("go.owl").load(reload=True, only_local=True)
      splitter = "/"
    print(DATASET, split)
    print(len(list(onto.object_properties())))
    relations = []
    with open(DATASET+"/"+split+"/"+DATASET+"_"+split+"_relations.txt") as f:
      for line in f:
        line = line.strip("\n").strip(">").strip("<")
        # print(line)
        relations.append(
          line.split(splitter)[-1]
        )
    print(len(relations))
    # relations = []
    for rel in onto.object_properties():
      name = str(rel).split(".")[-1]
      # print(name)
      # break
      if name in all_relations and name not in relations:
        try:
          print("destroying", rel)
          destroy_entity(rel)
        except:
          print("couldn't destroy ", name)
          # continue
      # break
        # print("deleting", name)
    filename = "raw/"+DATASET+"_"+split+".owl"
    print(onto, len(list(onto.properties())), list(onto.properties()))
    onto.save(file = filename, format = "rdfxml")

## Remove test axioms from RDF file to create training file

In [5]:
from owlready2 import *
from tqdm import tqdm

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
file = {
  "GALEN": (
    "raw/galen_{}.owl", # input file eg. galen_1_1.owl
    "GALEN/{}/{}.txt", # test file eg. test.txt
    "raw/galen_{}_without_test.owl", # output file eg. galen_1_1_without_test.owl
    "galen." # ontology prefix
  ),
  "GO": (
    "raw/GO_{}.owl",
    "GO/{}/{}.txt",
    "raw/go_{}_without_test.owl",
    "obo."
  ),
  "SNOMED": (
    "raw/SNOMED_{}.owl",
    "SNOMED/{}/{}.txt",
    "raw/snomed_{}_without_test.owl",
    "train."
  ),
  # "owl2bench": (
  #   "OWL2EL_5/{}/train.owl",
  #   "OWL2EL_5/{}/test.txt",
  #   "OWL2EL_5/OWL2EL_5_{}_without_test.owl"
  # )
}

In [53]:
def removeTest(onto, test, prefix="galen."):
  for a,b in tqdm(test):
    classes = onto.search(iri = "*"+a)
    if classes == []:
      continue
    for cls in classes:
      if str(cls) != prefix+a:
        continue
      cls.is_a = [subcls for subcls in cls.is_a if str(subcls) != prefix+b]
  return onto

In [54]:
for DATASET in ["GO"]:
  for split in ["1_1", "1_n", "n_n"]:
    onto = get_ontology(file[DATASET][0].format(split)).load(reload=True, only_local=True)
    # Get test pts
    test_pts = []
    sep = "/"
    with open(file[DATASET][1].format(split, "test")) as f:
      for line in f:
        line = line.strip("\n")
        a, b = line.split()
        c = a.strip().strip(">").split(sep)[-1]
        d = b.strip().strip(">").split(sep)[-1]
        test_pts.append((c,d))
    with open(file[DATASET][1].format(split, "valid")) as f:
      for line in f:
        line = line.strip("\n")
        a, b = line.split()
        c = a.strip().strip(">").split(sep)[-1]
        d = b.strip().strip(">").split(sep)[-1]
        test_pts.append((c,d))
    print(len(test_pts))
    onto = removeTest(onto, test_pts, file[DATASET][3])
    onto.save(file = file[DATASET][2].format(split), format = "rdfxml")

  0%|          | 2/26729 [00:00<36:45, 12.12it/s]

26729


  0%|          | 2/26729 [00:00<32:45, 13.60it/s]

26729


  0%|          | 2/36464 [00:00<43:09, 14.08it/s]

36464


100%|██████████| 36464/36464 [15:02:26<00:00,  1.48s/it]       
